In [ ]:
%pip install google_colab_selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random


In [ ]:
options = Options()
options.headless = False
options.add_argument('window_size=1920x1080')

driver = gs.Chrome(options=options)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
username = input("Enter your Letterboxd username: ")
# username = 'drewiepiebear'
website = f'https://letterboxd.com/{username}/watchlist/'

### Uncomment if you want to filter for genre

#genre = input("Enter the genre you want to watch: ")
# if genre == "all":
#   website = f'https://letterboxd.com/{username}/watchlist/'
# else:
#   website = f'https://letterboxd.com/{username}/watchlist/genre/{genre}/'


driver.get(website)

In [ ]:
#checks if there is any pagination
def check_next_page_exists(xpath):
  try:
    driver.find_element(By.XPATH, xpath)
  except:
    return False
  return True

xpath = '//div[@class="pagination"]'
if check_next_page_exists(xpath):
  #setting up variables for pagination
  pagination = driver.find_element(By.XPATH, '//div[@class="pagination"]')
  pages = pagination.find_elements(By.XPATH, './/li')
  next_page = pagination.find_element(By.XPATH, './/a[@class="next"]')
  last_page = int(pages[-1].text)
else:
  next_page = 1 #goes into while loop once just for the one page

movie_title = []
movie_release = []
current_page = 1


while next_page:
  #print("current page is " + str(current_page))

  # program was not going past the second page of pagination unless I repeated this in the while loop
  if check_next_page_exists(xpath):
    pagination = driver.find_element(By.XPATH, '//div[@class="pagination"]')

  container = driver.find_element(By.XPATH, '//ul[contains(@class , "poster-list")]')


  # dummy = pagination.get_attribute("innerHTML")
  # print(dummy)
  #container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class , "poster-list")]')))
  #movies = container.find_elements(By.XPATH, './li')
  movies = WebDriverWait(container,5).until(EC.presence_of_all_elements_located((By.XPATH, './li')))


  for movie in movies:
    #movie_title.append(movie.find_element(By.XPATH, './/div[@data-film-name]').get_attribute('data-film-name'))
    title_element = WebDriverWait(movie,5).until(EC.presence_of_element_located((By.XPATH, './/div[@data-film-name]'))).get_attribute('data-film-name')
    movie_title.append(title_element)
    #movie_release.append(movie.find_element(By.XPATH, './/div[@data-film-release-year]').get_attribute('data-film-release-year'))
    year_element = WebDriverWait(movie,5).until(EC.presence_of_element_located((By.XPATH, './/div[@data-film-release-year]'))).get_attribute('data-film-release-year')
    movie_release.append(year_element)

  try:
    #print("trying....")
    next_page = WebDriverWait(pagination, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[@class="next"]')))

    #next_page = pagination.find_element(By.XPATH, './/a[@class="next"]')
    next_page.click()
    current_page += 1

  except:
    #print("failed...")
    next_page = None

driver.quit()

In [ ]:
df_movies = pd.DataFrame({'title': movie_title, 'year': movie_release})
df_movies.to_csv('movies.csv', index=False)

In [ ]:
def pick_movie():
  def rng():
    return (random.randint(1,len(movie_title)) - 1)

  index = rng()
  title = movie_title[index]
  year = movie_release[index]
  return title, year

print(pick_movie())

('Bottle Rocket', '1994')
